In [2]:
import os
import face_recognition
from PIL import Image
import cv2
import math
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset,random_split,Subset
import torch.optim as optim

In [3]:
# Move model to GPU if available
device = torch.device("cuda:0")
print(device)

cuda:0


In [ ]:
#from video get frames
def capture_frames(video_path, output_folder,video_name, num_frames=10):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # print(total_frames)
    
    interval = math.floor(total_frames // num_frames)
    if interval == 0:
        interval = 1
    # print(interval)
    
    frame_count = 0
    success = True
    frame_number = 0
    while success and frame_number < total_frames:
        # Set the frame number to read
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        
        # Read a frame from the video
        success, frame = cap.read()
        
        # Check if its time to capture a frame
        if frame_number != 0 and interval !=0 and frame_number % interval == 0 and success:
            frame_path = os.path.join(output_folder, f"{video_name}_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
            # print(f"Saved frame {frame_count}")
            
            frame_count += 1

        frame_number += interval

    cap.release()
    print("Frame capture completed.")

def capture_frames_from_folder(video_folder, output_folder, num_frames=10):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all files in the video folder
    for file_name in os.listdir(video_folder):
        if file_name.endswith(('.mp4')):
            video_path = os.path.join(video_folder, file_name)
            video_name = os.path.splitext(file_name)[0]
            print(video_name)
            capture_frames(video_path, output_folder, video_name, num_frames)

In [ ]:
video_path = "D:/SUTD 8/50.021 Artificial Intelligence/proj/DeepfakeDetection3"
output_folder = "dataset/frames/DeepfakeDetection3"
num_frames_to_capture = 10

capture_frames_from_folder(video_path, output_folder, num_frames=num_frames_to_capture)

In [47]:

class DeepfakeDetectionDataset(Dataset):
    def __init__(self, root_dir, transform=None):

        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []

        for label, subdir in enumerate(['real', 'fake']):
            dir_path = os.path.join(self.root_dir, subdir)
            for file_name in os.listdir(dir_path):
                if file_name.endswith(('.jpg', '.jpeg', '.png')):  # Check file extension if necessary
                    img_path = os.path.join(dir_path, file_name)
                    image = face_recognition.load_image_file(img_path)
                    face_locations = face_recognition.face_locations(image)

                    # Only add the image if a suitable face is found
                    for top, right, bottom, left in face_locations:
                        if (bottom - top > 75) and (right - left > 75):
                            self.data.append(img_path)
                            self.labels.append(label)
                            break  # Found a suitable face, stop checking

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

            img_path = self.data[idx]
            image = face_recognition.load_image_file(img_path)

            face_locations = face_recognition.face_locations(image)
            top, right, bottom, left = face_locations[0]  # Use the first suitable face
            face_image = image[top:bottom, left:right]
            face_image = Image.fromarray(face_image)
            
            if self.transform:
                face_image = self.transform(face_image)

            label = self.labels[idx]
            return face_image, label
        else:
            return None  # Or handle accordingly

In [ ]:
# Define your transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Usage example
# dataset = DeepfakeDetectionDataset(root_dir='train', transform=transform)

In [1]:
# Assuming 'CustomImageDataset' and 'transformations' are defined
dataset = DeepfakeDetectionDataset(root_dir='train',
                            #  loader=DeepfakeDetectionDataset.loader,
                            #  extensions=['.jpg', '.jpeg', '.png'],
                             transform=transform)

# Split sizes
total_size = len(dataset)
train_size = int(0.8 * total_size)
test_size = int(0.1 * total_size)
val_size = total_size - train_size - test_size  # Remainder for validation
print(test_size)
print(train_size)
print(val_size)
total_samples = len(dataset)
indices = torch.randperm(total_samples)[:16]  # Randomly shuffle and select the first 100 indices

small_train_dataset = Subset(dataset, indices)
small_train_loader = DataLoader(small_train_dataset, batch_size=16, shuffle=True)

# Split the dataset
train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

# DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


NameError: name 'DeepfakeDetectionDataset' is not defined

In [49]:
def inspect_dataloader(dataloader):
    # Get the first batch
    images, labels = next(iter(dataloader))
    
    # Print the shapes and labels of the batch
    print(f"Batch shape: {images.shape}")
    print(f"Labels: {labels}")

print("Inspecting Training DataLoader:")
inspect_dataloader(train_loader)

print("\nInspecting Validation DataLoader:")
inspect_dataloader(val_loader)

print("\nInspecting Testing DataLoader:")
inspect_dataloader(test_loader)


Inspecting Training DataLoader:
Batch shape: torch.Size([4, 3, 224, 224])
Labels: tensor([1, 1, 1, 1])

Inspecting Validation DataLoader:
Batch shape: torch.Size([4, 3, 224, 224])
Labels: tensor([0, 0, 1, 1])

Inspecting Testing DataLoader:
Batch shape: torch.Size([4, 3, 224, 224])
Labels: tensor([1, 0, 1, 0])


In [54]:
# Load a pre-trained ResNet50 model
model = models.resnet50(weights="ResNet50_Weights.IMAGENET1K_V1")

# Modify the final layer for binary classification
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # Output for 2 classes: fake and real

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [56]:
def train_model(model, criterion, optimizer, dataloader, num_epochs=5):
    model.train()  # Set model to training mode
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_preds = 0
        
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            print(f"Input batch shape: {inputs.shape}")
            print(f"Labels batch shape: {labels.shape}")
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = correct_preds.double() / len(dataloader.dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')
        
    print('Training complete')
    return model

# Example usage:
# Adjust `dataloader` to your training DataLoader



In [57]:
train_model(model, criterion, optimizer, small_train_loader, num_epochs=5)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>